In [34]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# 对话缓存管理

## 基于内存的对话缓存：ConversationBufferMemory

In [35]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.save_context({"input": "你好啊"}, {"output": "你也好啊"})
memory.save_context({"input": "你再好啊"}, {"output": "你又好啊"})

memory.buffer_as_messages

[HumanMessage(content='hi!'),
 AIMessage(content="what's up?"),
 HumanMessage(content='你好啊'),
 AIMessage(content='你也好啊'),
 HumanMessage(content='你再好啊'),
 AIMessage(content='你又好啊')]

## 按窗口大保留对话缓存：ConversationBufferWindowMemory

In [36]:
from langchain.memory import ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(k=2)
window.save_context({"input": "第一轮问"}, {"output": "第一轮答"})
window.save_context({"input": "第二轮问"}, {"output": "第二轮答"})
window.save_context({"input": "第三轮问"}, {"output": "第三轮答"})
print(window.buffer_as_messages)

[HumanMessage(content='第二轮问'), AIMessage(content='第二轮答'), HumanMessage(content='第三轮问'), AIMessage(content='第三轮答')]


## 自动对历史信息做摘要：ConversationSummaryMemory

In [6]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    # buffer="The conversation is between a customer and a sales."
    buffer="以中文表示"
)
memory.save_context(
    {"input": "你好"}, {"output": "你好，我是你的AI助手。我能为你回答有关langchain的各种问题。"})

print(memory.load_memory_variables({}))

{'history': '\n人类问AI对人工智能的看法。AI认为人工智能是一种积极的力量，因为它能帮助人类发挥他们的全部潜力。人类向AI打招呼，AI回应说它是人类的AI助手，可以回答关于langchain的各种问题。'}


## 更多记忆类型

- ConversationTokenBufferMemory: 根据 Token 数限定 Memory 大小
  - https://python.langchain.com/docs/modules/memory/types/token_buffer
- VectorStoreRetrieverMemory: 将 Memory 存储在向量数据库中，根据用户输入检索回最相关的部分
  - https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory

# 在 LCEL 中直接使用对话缓存

<div class="alert-info" style="padding: 5px">
    <b>进一步扩展</b><p>
    这个例子中，使用了 langchain 的对话缓存类，同时也最大限度提供了代码逻辑的灵活性。<br>
    在此基础上，很容易继续添加持久化的逻辑。

langchain 中与记忆有关的两类工具都可以在下面的扩展中集成，如：

- langchain.memory.chat_message_histories 对话数据持久化相关的
- langchain.memory 按窗口大小、Token多少等建立缓存相关的
</div>

## 在一个带有记忆体的上下文中执行LCEL

现在，构造一个基础的提示语模板：

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])


实际上，我们的目标就是在LCEL运行逻辑中动态构造类似下面的提示语：

In [108]:
from langchain_core.messages import AIMessage

prompt.invoke({
    "history": [AIMessage(content="你好,我的名字是王小明")],
    "question": "你知道我现在的名字吗？"
}) 

ChatPromptValue(messages=[SystemMessage(content='你是一个AI助手'), AIMessage(content='你好,我的名字是王小明'), HumanMessage(content='你知道我现在的名字吗？')])

我们在LCEL运行环境中定义一个记忆体的共享变量，实现记忆体在各环节的提取和写入：<br>
完整代码如下：

In [111]:
# step1：定义窗口记忆的上下文变量
# ------------------------------
# 
from langchain.memory import ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(k=10)

# step2：保存用户输入到记忆（实现 RunnableLambda 版本）
# ------------------------------
#
from langchain_core.messages import BaseMessage
from typing import List, Iterator, AsyncIterator, Dict, Any

def memory_input(input: Any) -> str:
    window.chat_memory.add_user_message(input["question"])
    return window.buffer_as_messages

# step3：保存AI输出到记忆
# ------------------------------
#
def memory_output(messages: List[BaseMessage]) -> List[BaseMessage]:
    window.chat_memory.add_ai_message(messages)
    return(messages)

# step4：定义Chain
# ------------------------------
#
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

llm = ChatOpenAI()

chain = (
    RunnablePassthrough.assign(history=memory_input) 
    | prompt
    | llm
    | memory_output
    | StrOutputParser()
)

In [109]:
chain.invoke({"question":"我的名字是王小明"})

'你好，王小明！有什么问题我可以帮助你解决呢？'

In [110]:
# 也可以使用异步方法调用
await chain.ainvoke({"question":"你现在知道我的名字吗？"})

'是的，我知道你的名字是王小明。有什么问题我可以帮助你解决吗？'

## 支持流

In [43]:
# 保存AI输出
# 注意：如果要支持流式输出，就必须按照 RunnableGenerator 实现
def memory_output_stream(x: Iterator[Any]) -> Iterator[str]:
    chunks = ""
    try:
        for chunk in x:
            chunks += chunk.content
            yield chunk
    finally:
        window.chat_memory.add_ai_message(chunks)

In [44]:
chain = (
    RunnablePassthrough.assign(history=memory_input) 
    | prompt
    | llm
    | memory_output_stream
    | StrOutputParser()
)

In [45]:
# invoke
for chunk in chain.stream({"question": "我是小明"}):
    print(chunk, end="|", flush=True)

|你|好|，|小|明|！|有|什|么|可以|帮|到|你|的|吗|？||

## 支持异步流

In [55]:
async def memory_output_astream(x: AsyncIterator[Any]) -> AsyncIterator[str]:
    chunks = ""
    try:
        async for chunk in x:
            chunks += chunk.content
            yield chunk
    finally:
        window.chat_memory.add_ai_message(chunks)

In [56]:
chain = (
    RunnablePassthrough.assign(history=memory_input) 
    | prompt
    | llm
    | memory_output_astream
    | StrOutputParser()
)

In [57]:
# invoke
async for chunk in chain.astream({"question": "我是小明"}):
    print(chunk, end="|", flush=True)

|你|好|，|小|明|！|有|什|么|可以|帮|助|你|的|吗|？||

## 总结

从上面实现的方法中可以看到：

既要管理执行环境中的记忆体变量，又要实现 invoke / stream /astream 等方法，需要较多代码定义。<br>
你可能需要一个封装好的框架。

实际上，在langchain中已经有两种封装方案：
- 依靠 Chain 结构使用记忆体
- 使用 RunnableWithMessageHistory 使用记忆体


# 结合 LLMChain 中使用 Memory

<div class="alert-warning" style="padding:5px">
使用 LLMChain 非常方便，但没有实现 stream、astream、astream_events 等流式响应的方法。
</div>

## 使用 LLMChain

In [134]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个懂得多轮对话的助手"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])
# 注意：使用 `return_messages=True` 匹配 MessagesPlaceholder
# 注意：使用 `"chat_history"` 匹配提示语中的 MessagesPlaceholder
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, memory=memory)

In [135]:
conversation.invoke({"input": "我是孙小圣"})

{'input': '我是孙小圣',
 'chat_history': [HumanMessage(content='我是孙小圣'),
  AIMessage(content='你好，孙小圣！有什么可以帮助你的吗？')],
 'text': '你好，孙小圣！有什么可以帮助你的吗？'}

In [136]:
conversation.invoke({"input": "你知道我的名字吗？"})

{'input': '你知道我的名字吗？',
 'chat_history': [HumanMessage(content='我是孙小圣'),
  AIMessage(content='你好，孙小圣！有什么可以帮助你的吗？'),
  HumanMessage(content='你知道我的名字吗？'),
  AIMessage(content='是的，你刚刚告诉我你是孙小圣。有什么问题需要我帮忙解决吗？')],
 'text': '是的，你刚刚告诉我你是孙小圣。有什么问题需要我帮忙解决吗？'}

## 使用 ConversationChain

LLMChain的子类 ConversationChain 可以简化上述代码：

In [138]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

llm = ChatOpenAI()
conversation = ConversationChain(llm=llm)

In [139]:
conversation.invoke({"input": "我是孙小圣"})

{'input': '我是孙小圣',
 'history': '',
 'response': '你好，孙小圣先生！我是你的AI助手，很高兴认识你。有什么可以帮助你的吗？'}

In [140]:
conversation.invoke({"input": "你知道我的名字吗？"})

{'input': '你知道我的名字吗？',
 'history': 'Human: 我是孙小圣\nAI: 你好，孙小圣先生！我是你的AI助手，很高兴认识你。有什么可以帮助你的吗？',
 'response': '当然知道啦，你刚才告诉我你是孙小圣先生啊！你有什么想要了解或者讨论的吗？'}

## LLMChain的不足

<div class="alert alert-info">
    <b>Chain 相关类是一个遗留技术分支</b><br>
    LLMChain 属于 Chain 技术生态的一种是实现。<br>
    这类实现使用起来都非常方便，但没有实现 stream、astream、astream_events 等方法。<br><br>
    Chain 技术生态是一个遗留技术分支，新的功能模块应当尽量考虑采用 LCEL 或者 langgraph 两个技术分支。<br>
    但 Chain 已经实现了包括 LLMChain 在内的非常丰富的生态成果。因此，langchain 官方打算继续维护 Chain 这个技术分支，直到给出 LCEL 的替代版本。

In [142]:
for chunk in conversation.stream({"input": "你知道我的本领是什么吗？"}):
    print(chunk)

{'input': '你知道我的本领是什么吗？', 'history': 'Human: 我是孙小圣\nAI: 你好，孙小圣先生！我是你的AI助手，很高兴认识你。有什么可以帮助你的吗？\nHuman: 你知道我的名字吗？\nAI: 当然知道啦，你刚才告诉我你是孙小圣先生啊！你有什么想要了解或者讨论的吗？', 'response': '据我的记录显示，您的本领包括编程、数据分析和人工智能等领域。您在这些领域有很深的造诣，是一个非常优秀的专家。您还有其他特长吗？我很乐意了解更多关于您的信息。'}


# 使用 RunnableWithMessageHistory 管理记忆

<div class="alert-warning" style="padding:5px">
可以全面支持 LCEL 的所有方法，包括 stream 等，但不能灵活地管理对话窗口。
</div>

In [122]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手，每次返回20个以内的字即可"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
llm = ChatOpenAI()

# 构建链
chain = prompt | llm

In [45]:
# 现在要运行链，需要三个输入参数
chain.input_schema()

PromptInput(ability=None, history=None, input=None)

## 在内存中保存

**ChatMessageHistory**

In [123]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [125]:
# 第 1 次
with_message_history.invoke(
    {"ability": "数学", "input": "三角函数是什么意思?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='三角函数是一类描述角度和三角形边长关系的数学函数。')

In [128]:
# 第 2 次
for chunk in with_message_history.stream(
    {"ability": "数学", "input": "可以用小学三年级小朋友能听懂的话解释吗？"},
    config={"configurable": {"session_id": "abc123"}},
):
    print(chunk.content, end="|", flush=True)

|三|角|函数|就|是|一|种|数|学|工|具|，|帮|助|我们|计|算|和|描述|三|角|形|里|面|的|角|度|和|边|长|关|系|。|就|像|用|魔|法|棒|一|样|，|帮|我们|解|决|数|学|难|题|。||

In [99]:
# 此时 store 中多了两条消息历史
store

{'abc123': ChatMessageHistory(messages=[HumanMessage(content='三角函数是什么意思?'), AIMessage(content='三角函数是一类描述角与边之间关系的数学函数，包括正弦、余弦、正切、余切、正割和余割等。这些函数在几何学、物理学、工程学等领域中有广泛的应用。'), HumanMessage(content='可以用小学三年级小朋友能听懂的话解释吗？'), AIMessage(content='三角函数就是一种用来帮助我们计算三角形内角和边之间关系的工具。比如，我们可以用它来帮助我们求出三角形中的角度大小或者边长等信息。')])}

In [10]:
# 如果切换了 session_id 情况就会不同
with_message_history.invoke(
    {"ability": "数学", "input": "可以用小学三年级小朋友能听懂的话解释吗？"},
    config={"configurable": {"session_id": "abc456"}},
)

AIMessage(content='当我们在数学里谈到"20以内"，就是指所有小于等于20的数字哦！比如1、2、3、4、5、6、7、8、9、10、11、12、13、14、15、16、17、18、19和20。')

## 自定义对话历史的键值

In [82]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="用户ID",
            description="用户唯一标识",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="对话ID",
            description="对话唯一标识",
            default="",
            is_shared=True,
        ),
    ],
)

In [83]:
# with_message_history 此时的配置项是 user_id 和 conversation_id
with_message_history.config_specs

[ConfigurableFieldSpec(id='conversation_id', annotation=<class 'str'>, name='对话ID', description='对话唯一标识', default='', is_shared=True, dependencies=None),
 ConfigurableFieldSpec(id='user_id', annotation=<class 'str'>, name='用户ID', description='用户唯一标识', default='', is_shared=True, dependencies=None)]

In [84]:
# 此时调用就要提供 user_id 和 conversation_id
with_message_history.invoke(
    {"ability": "math", "input": "你好"},
    config={"configurable": {"user_id": "user-123", "conversation_id": "conv-1"}},
)

AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')

In [85]:
# 查看对话历史记录，键值已经变为由 user_id 和 conversation_id 的值构成的元组
store

{('user-123',
  'conv-1'): ChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')])}

In [89]:
# 实际上 with_message_history 对象也已经通过 _merge_configs 函数动态绑定了对话历史
with_message_history._merge_configs({
    "configurable": {
        "user_id": "user-123", 
        "conversation_id": "conv-1"
    }
})
# 下面的输出结果在每次调用 invoke 时会作为 config 参数被携带

{'configurable': {'user_id': 'user-123',
  'conversation_id': 'conv-1',
  'message_history': ChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content='你好，请问有什么数学问题我可以帮助您解决呢？')])}}

## 使用 output_messages_key

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOpenAI()})

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

## 自定义记忆持久化

```python
# 基类要求
class BaseChatMessageHistory(ABC):
    messages: List[BaseMessage]
    """必须重载：读取消息"""

    @abstractmethod
    def add_message(self, message: BaseMessage) -> None:
        """必须重载：增加消息历史"""
        raise NotImplementedError()

    @abstractmethod
    def clear(self) -> None:
        """必须重载：清除所有消息"""

```

### 基于内存的记忆管理实现

```python
from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain.memory import ConversationBufferWindowMemory

class ChatMessageHistory(BaseChatMessageHistory, BaseModel):
    """基于内存的消息历史管理，直接存储在内存列表中"""

    messages: List[BaseMessage] = Field(default_factory=list)    

    def __init__(self, session_id: str, k: int):
        self.file_path = Path(file_path)
    
    def add_message(self, message: BaseMessage) -> None:
        """增加新消息"""
        self.messages.append(message)

    def clear(self) -> None:
        self.messages = []
```

### 基于文件的记忆管理实现

这里是更完整的例子：[https://github.com/langchain-ai/langserve/blob/main/examples/chat_with_persistence_and_user/server.py]

```python
import json
import logging
from pathlib import Path
from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import (
    BaseMessage,
    messages_from_dict,
    messages_to_dict,
)

logger = logging.getLogger(__name__)


class FileChatMessageHistory(BaseChatMessageHistory):
    """基于文件管理消息历史的示范，所有消息存储在本地JSON文件中

    参数:
        file_path: 保存JSON文件的路径
    """

    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        if not self.file_path.exists():
            self.file_path.touch()
            self.file_path.write_text(json.dumps([]))

    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """从 JSON 文件提取历史消息"""
        items = json.loads(self.file_path.read_text())
        messages = messages_from_dict(items)
        return messages

    def add_message(self, message: BaseMessage) -> None:
        """将新消息增加到本地 JSON 文件"""
        messages = messages_to_dict(self.messages)
        messages.append(messages_to_dict([message])[0])
        self.file_path.write_text(json.dumps(messages))

    def clear(self) -> None:
        """清理所有消息历史"""
        self.file_path.write_text(json.dumps([]))

```